In [1]:
from model import MLP
import onnxruntime as ort
import torch
import numpy as np

/home/free4ky/projects/museum_search/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
inputs = torch.randn((4, 1408), dtype=torch.float32) 

In [3]:
oinputs = {
    "classification_input": inputs.numpy().astype(np.float32)
}

In [4]:
sess = ort.InferenceSession('mlp.onnx', providers=['CPUExecutionProvider'])

In [5]:
ores = sess.run(None, oinputs)

In [6]:
ores[0]

array([[-0.90238297, -2.3921878 , -0.17438602, -1.6931089 , -1.0652775 ,
         0.5103612 ,  0.14319254,  4.1414857 , -2.9492524 , -3.933328  ,
         5.105617  , -1.7843027 ,  1.4222983 , -3.2117186 , -3.8466911 ],
       [-1.8671016 , -2.7724364 ,  1.284509  ,  9.238093  , -2.8287537 ,
        -2.3467941 , -2.1185303 , -1.2867466 , -1.4339646 , -4.735503  ,
         4.0909805 , -0.38293174, -3.032908  , -2.2355843 , -6.3741193 ],
       [-8.134947  , -7.4787707 ,  1.0061442 , -4.7199445 ,  2.5116198 ,
        -3.2882864 , -1.979547  , -5.6660323 ,  4.9119496 , -4.5984674 ,
         1.6981157 ,  9.749955  , -1.078701  , -9.464261  , -4.188462  ],
       [-2.0295663 , -0.6436125 , -4.2777104 ,  1.2209531 , -3.5920734 ,
        -6.751383  , -3.3041148 , -8.608288  ,  7.51472   , -5.3224564 ,
         3.4173543 ,  6.618896  , -3.155294  , -6.021234  , -7.7269797 ]],
      dtype=float32)

In [7]:
model = MLP()
model.load_state_dict(torch.load('./mlp_dict.pth'))
# model = model.half()

<All keys matched successfully>

In [8]:
with torch.no_grad():
    torch_res = model(inputs)

In [10]:
torch_res

tensor([[ -0.5378,  -2.5193,   1.1920,  -1.3169,  -1.3155,   0.2960,   0.4848,
           5.7498,  -3.7689,  -3.0399,   2.9376,  -1.7302,   1.1243,  -3.6886,
          -3.7673],
        [ -1.5204,  -5.5830,   2.6705,  13.5325,  -5.5915,  -0.5357,  -4.8715,
          -4.3307,  -1.1725,  -3.9202,   4.4362,  -1.5446,  -3.0866,   0.3739,
          -8.2610],
        [ -8.6327,  -5.1015,   0.1752,  -4.4517,   2.9044,  -3.5610,   0.6088,
          -4.0003,   5.3174,  -6.0430,   2.3320,   6.2603,  -1.4614,  -9.2696,
          -3.8219],
        [ -2.0227,  -1.1279,  -4.1546,   2.2820,  -6.3994,  -6.4794,  -5.5603,
         -13.0608,  12.8246,  -5.5082,   5.0840,   6.1143,  -0.5989,  -7.3750,
         -10.2402]])

In [11]:
ores[0]

array([[-0.90238297, -2.3921878 , -0.17438602, -1.6931089 , -1.0652775 ,
         0.5103612 ,  0.14319254,  4.1414857 , -2.9492524 , -3.933328  ,
         5.105617  , -1.7843027 ,  1.4222983 , -3.2117186 , -3.8466911 ],
       [-1.8671016 , -2.7724364 ,  1.284509  ,  9.238093  , -2.8287537 ,
        -2.3467941 , -2.1185303 , -1.2867466 , -1.4339646 , -4.735503  ,
         4.0909805 , -0.38293174, -3.032908  , -2.2355843 , -6.3741193 ],
       [-8.134947  , -7.4787707 ,  1.0061442 , -4.7199445 ,  2.5116198 ,
        -3.2882864 , -1.979547  , -5.6660323 ,  4.9119496 , -4.5984674 ,
         1.6981157 ,  9.749955  , -1.078701  , -9.464261  , -4.188462  ],
       [-2.0295663 , -0.6436125 , -4.2777104 ,  1.2209531 , -3.5920734 ,
        -6.751383  , -3.3041148 , -8.608288  ,  7.51472   , -5.3224564 ,
         3.4173543 ,  6.618896  , -3.155294  , -6.021234  , -7.7269797 ]],
      dtype=float32)

In [45]:
sum(abs(torch.nn.functional.softmax(torch_res, dim=-1) - torch.nn.functional.softmax(torch.from_numpy(ores[0]), dim=-1)))

tensor([1.8921e-03, 3.3531e-03, 5.5859e-01, 2.4700e-04, 1.3411e-05, 1.0717e-04,
        3.7432e-05, 4.7632e-01, 2.0943e-03, 4.5836e-05, 3.8314e-04, 6.8054e-02,
        1.4046e-02, 7.5698e-06, 8.1062e-06], dtype=torch.float16)

tensor([[1.0000e+00, 0.0000e+00, 0.0000e+00, 5.9605e-07, 0.0000e+00, 5.9605e-08,
         0.0000e+00, 0.0000e+00, 9.3579e-06, 5.9605e-08, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 3.6359e-06, 0.0000e+00],
        [1.6451e-05, 1.4937e-04, 5.5176e-01, 2.4080e-05, 7.6890e-06, 1.2279e-05,
         1.4246e-05, 4.3481e-01, 2.3842e-07, 2.3842e-07, 5.0187e-05, 8.9340e-03,
         4.1313e-03, 5.9605e-08, 2.3842e-06],
        [4.9114e-05, 9.8682e-01, 7.7438e-03, 2.2173e-05, 1.7881e-07, 3.5763e-07,
         3.5167e-06, 9.3579e-06, 1.7242e-03, 1.0133e-06, 5.1260e-06, 3.6869e-03,
         1.5497e-06, 2.5630e-06, 1.4007e-05],
        [1.0389e-04, 6.9618e-04, 9.9121e-01, 5.9557e-04, 4.4107e-06, 7.9572e-05,
         4.9710e-05, 4.5657e-04, 7.0858e-04, 5.1320e-05, 1.4818e-04, 5.6915e-03,
         1.8442e-04, 2.0862e-06, 4.8280e-06]], dtype=torch.float16)